In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import json
import re
from bs4 import BeautifulSoup as bs

In [2]:
# Doesn't work, 403 forbidden response.
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
#     "Accept-Encoding": "gzip, deflate, br",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
#     "Connection": "keep-alive",
#     "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
# }

# response = httpx.get("https://www.indeed.com/jobs?q=data+science&l=remote&vjk=96bf68966c7569d7", header=HEADERS)
# print(response)

In [3]:
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://www.indeed.com/jobs?q=data+science&l=remote")
time.sleep(1)

soup = bs(driver.page_source)

In [4]:
results = soup.find(id="jobsearch-Main")

In [5]:
job_elements = results.find_all("div", class_="slider_item")

In [5]:
from pyppeteer import launch

In [ ]:
async def main():
    browser = await launch()
    page = await browser.newPage()
    await page.goto("https://www.indeed.com/jobs?q=data+science&l=remote")
    html = await page.content()
    return html

In [7]:
basePath = "https://www.indeed.com"
d = {}
# Currently using link_element_id as unique ID.  I am fairly sure it's unique - at least it has been from what I've seen so far.
for job_element in job_elements[:1]:
    link_element_id = job_element.find_all("a")[0]["id"]

    d[link_element_id] = {}
    d[link_element_id]["title"] = job_element.find("h2", class_="jobTitle").text.strip()
    d[link_element_id]['company'] = job_element.find("span", class_="companyName").text.strip()
    d[link_element_id]['posted'] = job_element.find("span", class_="date").next_element.next_element.next_element
    try:
        d[link_element_id]['location'] = job_element.find("div", class_="companyLocation").text.strip()
    except AttributeError: 
        d[link_element_id]['location'] = "Not Listed"
    try:
        d[link_element_id]['salary'] = job_element.find("div", class_="metadata").text.strip()
    except AttributeError:
        d[link_element_id]['salary'] = "Not Listed"
    # date_element = job_element.find("div", class_="visually-hidden")  # not useful as-is.  figure out if this is possible to find at all

    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="{link_element_id}"]'))).click()

    time.sleep(2)
    temp_results = results.find("div", id='jobDescriptionText')
    # print(d[link_element_id]['title'], d[link_element_id]['company'])
    d[link_element_id]['desc'] = temp_results.text.strip()

# print(title_element.text.strip())
# print(company_element.text.strip())
# print(location_element.text.strip())
# print(salary_element.text.strip())
# print(jobsnippet_element.text.strip())
# print(full_desc_element)
# print("\n")
# time.sleep(10)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
d

{'sj_1d2fe01649ff8b16': {'title': 'Sr. Architect – Data , AI, Information Architecture',
  'company': 'The Travelers Companies, Inc.',
  'posted': 'Posted 30+ days ago',
  'location': 'Remote in Hartford, CT 06101',
  'salary': '$130,600 - $215,400 a year',
  'desc': "It's fun to work in a company where people truly BELIEVE in what they're doing!\nWe're committed to bringing passion and customer focus to the business.\nJob purpose\nCelink is rapidly building proprietary servicing system on AWS and automating manual servicing processes. Data is a critical part of our business and influences every aspect of the work that we do. The Data Management (DM) team is hiring a Data Scientist to focus on data analytical efforts. This role must ensure the successful completion of multiple statistical/analytical models that can be applied on Celink Data Lake that will provide insights for business operations.\nAs a Data Scientist, you will analyze complex mortgage and servicing operations data sets

Once I can get the full job desc, I will want to do the following:
For each job, create a JSON object for it.  Each JSON object will have a structure like

{ "title":job_title, "company":company_name, "location":job_location, "salary":job_salary, "snippet":job_desc_snippet, "desc":full_job_desc, "tags":search_terms_used_to_find_job}

I want a way to go through and if the same job is repeated, add on to the tags in the proper json.  Can do this by instead building it as a dict, appending to the "tags" key so then search_terms_used_to_find_job is instead a list thereof, and the list is appended with the new search terms before being json-ified.

Need to figure out if I can / how to get the date posted.

Need to figure out if the job ID on indeed is unique.  If I can't figure it out, may need my own unique ID for each

Probably remove the job desc snippet since I have full descs now

Need to figure out how to go through all pages

Need to add tags for search params

Can use: https://scrapfly.io/blog/how-to-scrape-indeedcom/
To see how to better optimize this search.  Both for making searching more dynamic as well as using async IO to speed it up